In [1]:
from vacunacion_regional.entrypoints import api
from vacunacion_regional.data_management.postprocessing import process_model_variables, get_mapped_variables
import pandas as pd

params, mappings = api.get_from_data_parameters('./data/vaccines.json')
model = api.obtain_model(params)
model.read("out.mps")
model.optimize()
mapped_variables = get_mapped_variables(model)
variables = process_model_variables(mapped_variables)

Academic license - for non-commercial use only - expires 2021-08-02
Using license file D:\Users\gabri\gurobi.lic
Read MIP start from file out.mst
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 8705 rows, 6100 columns and 43100 nonzeros
Model fingerprint: 0x9afb8399
Variable types: 500 continuous, 5600 integer (2800 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+12]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+12]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.

User MIP start produced solution with objective 239989 (0.03s)
Loaded user MIP start with objective 239989

Presolve removed 5633 rows and 2419 columns
Presolve time: 0.07s
Presolved: 3072 rows, 3681 columns, 16272 nonzeros
Variable types: 485 continuous, 3196 integer (2796 binary)

Root relaxation:

In [2]:
from vacunacion_regional.data_management.postprocessing import generate_tables

tables = generate_tables(model, mappings=mappings, save=True)




In [3]:
tables.keys()

dict_keys(['camion_n_en_comuna_c_dia_d', 'vacunas_camion_dia', 'personas_vacunadas_comuna_dia', 'porcentajes_comuna_dia', 'promedio_vacunacion_dia', 'comuna_critica'])

In [4]:
new_table = tables["camion_n_en_comuna_c_dia_d"].drop(["dia", "camion"], axis=1).groupby(by=["comuna"]).agg(['sum'])
new_table

,value
,sum
comuna,
Arica,586.0
Camarones,3.0
General Lagos,3.0
Putre,8.0


In [5]:
print(new_table.to_latex())

\begin{tabular}{lr}
\toprule
{} &  value \\
{} &    sum \\
comuna        &        \\
\midrule
Arica         &  586.0 \\
Camarones     &    3.0 \\
General Lagos &    3.0 \\
Putre         &    8.0 \\
\bottomrule
\end{tabular}



In [6]:
tables["vacunas_camion_dia"].head()

,camion,comuna,dia,value
0,0,Arica,0,1000.0
1,0,Arica,1,1000.0
2,0,Arica,2,1000.0
3,0,Arica,3,1000.0
4,0,Arica,4,1000.0


In [15]:
new_table = tables["vacunas_camion_dia"].drop(["dia"], axis=1).groupby(by=["camion", "comuna"]).agg(['sum'])
new_table

value
                           sum
camion comuna                 
0      Arica          100000.0
       Camarones           0.0
       General Lagos       0.0
       Putre               0.0
1      Arica           94000.0
       Camarones        1000.0
       General Lagos    1000.0
       Putre            4000.0
2      Arica           97000.0
       Camarones        1000.0
       General Lagos       0.0
       Putre            2000.0
3      Arica           98000.0
       Camarones        1000.0
       General Lagos       0.0
       Putre            1000.0
4      Arica           98000.0
       Camarones           0.0
       General Lagos    1000.0
       Putre            1000.0
5      Arica           99000.0
       Camarones           0.0
       General Lagos    1000.0
       Putre               0.0

In [16]:
print(new_table.to_latex())

\begin{tabular}{llr}
\toprule
  &       &     value \\
  &       &       sum \\
camion & comuna &           \\
\midrule
0 & Arica &  100000.0 \\
  & Camarones &       0.0 \\
  & General Lagos &       0.0 \\
  & Putre &       0.0 \\
1 & Arica &   94000.0 \\
  & Camarones &    1000.0 \\
  & General Lagos &    1000.0 \\
  & Putre &    4000.0 \\
2 & Arica &   97000.0 \\
  & Camarones &    1000.0 \\
  & General Lagos &       0.0 \\
  & Putre &    2000.0 \\
3 & Arica &   98000.0 \\
  & Camarones &    1000.0 \\
  & General Lagos &       0.0 \\
  & Putre &    1000.0 \\
4 & Arica &   98000.0 \\
  & Camarones &       0.0 \\
  & General Lagos &    1000.0 \\
  & Putre &    1000.0 \\
5 & Arica &   99000.0 \\
  & Camarones &       0.0 \\
  & General Lagos &    1000.0 \\
  & Putre &       0.0 \\
\bottomrule
\end{tabular}



In [8]:
new_table = tables["personas_vacunadas_comuna_dia"].drop("comuna", axis=1).groupby(by=["dia"]).agg(['sum'])
new_table

,value
,sum
dia,
0,2400.0
1,2400.0
2,2400.0
3,2400.0
4,2400.0
...,...
95,2400.0
96,2400.0


In [23]:
import numpy as np
bins = [-np.inf, 20, 40, 60, 80, np.inf]
names = ['0-19', '20-39', '40-59', '60-79', '80-99']
new_table = tables["porcentajes_comuna_dia"].copy()
new_table['dias'] = pd.cut(new_table['dia'], bins, labels=names)
new_table = new_table.drop(['dia'], axis=1).groupby(['dias', 'comuna']).agg(["max", "min", "mean"]).round(2)
new_table

value                  
                      max   min  mean   std
dias  comuna                               
0-19  Arica          0.47  0.35  0.41  0.04
      Camarones      0.78  0.39  0.57  0.20
      General Lagos  1.00  0.11  0.95  0.19
      Putre          0.61  0.23  0.46  0.09
20-39 Arica          0.59  0.47  0.53  0.04
      Camarones      0.78  0.78  0.78  0.00
      General Lagos  1.00  1.00  1.00  0.00
      Putre          0.71  0.61  0.63  0.04
40-59 Arica          0.72  0.60  0.66  0.04
      Camarones      0.78  0.78  0.78  0.00
      General Lagos  1.00  1.00  1.00  0.00
      Putre          0.80  0.71  0.77  0.05
60-79 Arica          0.84  0.73  0.78  0.04
      Camarones      0.97  0.78  0.84  0.09
      General Lagos  1.00  1.00  1.00  0.00
      Putre          0.90  0.80  0.83  0.04
80-99 Arica          0.96  0.85  0.91  0.04
      Camarones      0.97  0.97  0.97  0.00
      General Lagos  1.00  1.00  1.00  0.00
      Putre          1.00  0.90  0.93  0.05

In [24]:
print(new_table.to_latex())

\begin{tabular}{llrrrr}
\toprule
      &       & \multicolumn{4}{l}{value} \\
      &       &   max &   min &  mean &   std \\
dias & comuna &       &       &       &       \\
\midrule
0-19 & Arica &  0.47 &  0.35 &  0.41 &  0.04 \\
      & Camarones &  0.78 &  0.39 &  0.57 &  0.20 \\
      & General Lagos &  1.00 &  0.11 &  0.95 &  0.19 \\
      & Putre &  0.61 &  0.23 &  0.46 &  0.09 \\
20-39 & Arica &  0.59 &  0.47 &  0.53 &  0.04 \\
      & Camarones &  0.78 &  0.78 &  0.78 &  0.00 \\
      & General Lagos &  1.00 &  1.00 &  1.00 &  0.00 \\
      & Putre &  0.71 &  0.61 &  0.63 &  0.04 \\
40-59 & Arica &  0.72 &  0.60 &  0.66 &  0.04 \\
      & Camarones &  0.78 &  0.78 &  0.78 &  0.00 \\
      & General Lagos &  1.00 &  1.00 &  1.00 &  0.00 \\
      & Putre &  0.80 &  0.71 &  0.77 &  0.05 \\
60-79 & Arica &  0.84 &  0.73 &  0.78 &  0.04 \\
      & Camarones &  0.97 &  0.78 &  0.84 &  0.09 \\
      & General Lagos &  1.00 &  1.00 &  1.00 &  0.00 \\
      & Putre &  0.90 &  0.80 &  

In [10]:
import numpy as np
bins = [-np.inf, 20, 40, 60, 80, np.inf]
names = ['0-19', '20-39', '40-59', '60-79', '80-99']
new_table = tables["promedio_vacunacion_dia"].copy()
new_table['dias'] = pd.cut(new_table['dia'], bins, labels=names)
new_table = new_table.drop(['dia'], axis=1).groupby(['dias']).agg(["max", "min", "mean", "std"])
new_table

value                              
            max       min      mean       std
dias                                         
0-19   0.713251  0.267205  0.598574  0.104073
20-39  0.768860  0.714843  0.733531  0.016313
40-59  0.824462  0.770452  0.799832  0.020557
60-79  0.928587  0.826054  0.861674  0.040451
80-99  0.982604  0.930179  0.952012  0.019316

In [11]:
print(new_table.to_latex())

\begin{tabular}{lrrrr}
\toprule
{} & \multicolumn{4}{l}{value} \\
{} &       max &       min &      mean &       std \\
dias  &           &           &           &           \\
\midrule
0-19  &  0.713251 &  0.267205 &  0.598574 &  0.104073 \\
20-39 &  0.768860 &  0.714843 &  0.733531 &  0.016313 \\
40-59 &  0.824462 &  0.770452 &  0.799832 &  0.020557 \\
60-79 &  0.928587 &  0.826054 &  0.861674 &  0.040451 \\
80-99 &  0.982604 &  0.930179 &  0.952012 &  0.019316 \\
\bottomrule
\end{tabular}



In [17]:
import numpy as np
bins = [-np.inf, 24, 49, 74, np.inf]
names = ['0-24', '25-49', '50-74', '75-99']
new_table = tables["comuna_critica"].copy()
new_table['dias'] = pd.cut(new_table['dia'], bins, labels=names)
new_table = new_table.drop(['dia'], axis=1).groupby(['dias', 'comuna']).agg(["sum"])
new_table

value
                      sum
dias  comuna             
0-24  Arica          25.0
      Camarones      11.0
      General Lagos   1.0
      Putre          25.0
25-49 Arica          25.0
      Camarones       4.0
      General Lagos   0.0
      Putre          25.0
50-74 Arica          25.0
      Camarones      25.0
      General Lagos   0.0
      Putre          25.0
75-99 Arica          25.0
      Camarones       6.0
      General Lagos   0.0
      Putre          19.0

In [18]:
print(new_table.to_latex())

\begin{tabular}{llr}
\toprule
      &       & value \\
      &       &   sum \\
dias & comuna &       \\
\midrule
0-24 & Arica &  25.0 \\
      & Camarones &  11.0 \\
      & General Lagos &   1.0 \\
      & Putre &  25.0 \\
25-49 & Arica &  25.0 \\
      & Camarones &   4.0 \\
      & General Lagos &   0.0 \\
      & Putre &  25.0 \\
50-74 & Arica &  25.0 \\
      & Camarones &  25.0 \\
      & General Lagos &   0.0 \\
      & Putre &  25.0 \\
75-99 & Arica &  25.0 \\
      & Camarones &   6.0 \\
      & General Lagos &   0.0 \\
      & Putre &  19.0 \\
\bottomrule
\end{tabular}



In [48]:
new_table = tables["porcentajes_comuna_dia"].copy()
final = (new_table[new_table['dia'] == 99]['value']).std()
final

0.016956366346322846

In [49]:
inicial = pd.DataFrame({"comuna": params.comunas, "porcentaje_vacunados":np.array(params.poblacion_vacunada)/np.array(params.poblacion_objetivo)}).std()['porcentaje_vacunados']
inicial

0.12744621467917308

In [65]:
print(f"La desviación estandar mejoro un {round(100*abs(inicial - final)/inicial, 3)} %")

La desviación estandar mejoro un 86.695 %


In [73]:
print(f"Porcentaje de vacunacion final es de {100 * tables['promedio_vacunacion_dia'][tables['promedio_vacunacion_dia']['dia'] == 99].round(5)['value'][99]} %")

Porcentaje de vacunacion final es de 98.26 %


In [76]:
print(f"Porcentaje de vacunacion inicial es de {pd.Series({'porcentaje_vacunados': 100 * np.array(params.poblacion_vacunada).sum()/np.array(params.poblacion_objetivo).sum()}).round(3)['porcentaje_vacunados']} %")

Porcentaje de vacunacion inicial es de 34.641 %
